<a href="https://colab.research.google.com/github/jennyschilling/DataProject1_DS2002/blob/main/noSQL_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# integrating data from a NoSQL database - reviews in product dimension - denormalization



1. Fetch reviews from MongoDB
2. Transform reviews into a DataFrame
3. Add columns like 'review_text' and 'review_rating' to the 'product_dim' table in your SQL databases
4. Update the 'product_dim' with the new review data



In [ ]:
import pymongo
import pandas as pd
import pymysql
from faker import Faker

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["local"]

In [ ]:
reviews_collection = db["product_reviews"]

for i in range(110): # generates 110 random reviews
    review = {
        "product_key": i+1,
        "reviewer": fake.name(),
        "review_text": fake.paragraph(nb_sentences=3),
        "rating": fake.random_int(min=1, max=5)
    }
    reviews_collection.insert_one(review)

In [ ]:
results = reviews_collection.find()

In [ ]:
df = pd.DataFrame(list(results))